# Installing required libraries

In [ ]:
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install seaborn
# %pip install scipy

In [ ]:
from preprocessor.sklearn_preprocessor import preprocess_data

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = './data/'
data = pd.read_csv(path + 'train.csv')

In [ ]:
print(f'Số lượng dòng của data: {len(data)}')
print(data.columns)

In [ ]:
data.head()

In [ ]:
data.info()

# I. Preprocessing Data

## 1. Cleaning our data

-

In [ ]:
data.dropna(how='all')
data.drop_duplicates()

data["Engine"] = data["Engine"].str.replace(' cc', '', regex=False).astype(float)

data[['Max Power BHP', 'Max Power RPM']] = data['Max Power'].str.extract(r'(\d+)\s*bhp\s*@\s*(\d+)\s*rpm')
data['Max Power BHP'] = pd.to_numeric(data['Max Power BHP'], errors='coerce')
data['Max Power RPM'] = pd.to_numeric(data['Max Power RPM'], errors='coerce')

data[['Max Torque Nm', 'Max Torque RPM']] = data['Max Torque'].str.extract(r'(\d+)\s*Nm\s*@\s*(\d+)\s*rpm')
data['Max Torque Nm'] = pd.to_numeric(data['Max Torque Nm'], errors='coerce')
data['Max Torque RPM'] = pd.to_numeric(data['Max Torque RPM'], errors='coerce')

data = data.drop('Max Power', axis=1)
data = data.drop('Max Torque', axis=1)

spltio


In [ ]:
data["Drivetrain"].value_counts()

In [ ]:
data["Fuel Type"].value_counts()

In [ ]:
# Get the unique values before making changes
print("Unique values before:", data["Fuel Type"].unique())

# Replace less common fuel types with 'Others'
rare_fuel_types = ['Electric', 'LPG', 'Hybrid', 'CNG + CNG', 'Petrol + LPG']
data["Fuel Type"] = data["Fuel Type"].apply(lambda x: 'Others' if x in rare_fuel_types else x)

# Check the unique values after making changes
print("Unique values after:", data["Fuel Type"].unique())

In [ ]:
# Apply preprocessing
train_df, val_df, test_df, preprocessor = preprocess_data(
    data=data, 
    save_path='./processed_data/'
)

In [ ]:
train_df.head()

In [ ]:
### Insert feature engineer code 
### For simplicity, just choose all features, drop Make, Model and standard hypothesis model
train_df = train_df.drop(['Make', 'Model', 'Transmission', 'Location', 'Color', 'Seller Type'], axis=1)
val_df = val_df.drop(['Make', 'Model', 'Transmission', 'Location', 'Color', 'Seller Type'], axis=1)
test_df = test_df.drop(['Make', 'Model', 'Transmission', 'Location', 'Color', 'Seller Type'], axis=1)


# Check if there are any columns with object type in the transformed dataframes
print("Train DataFrame Object Types:", train_df.select_dtypes(include=['object']).columns.tolist())
print("Val DataFrame Object Types:", val_df.select_dtypes(include=['object']).columns.tolist())
print("Test DataFrame Object Types:", test_df.select_dtypes(include=['object']).columns.tolist())

# If there are object columns, convert them to numeric
for df in [train_df, val_df, test_df]:
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col])

# Verify the conversion worked
print("\nAfter conversion:")
print("Train DataFrame Types:", train_df.dtypes.value_counts())
print("Val DataFrame Types:", val_df.dtypes.value_counts())
print("Test DataFrame Types:", test_df.dtypes.value_counts())

In [ ]:
from modelling.Model import StandardRegression
model = StandardRegression()
train_df = model.transform_features(train_df)
val_df = model.transform_features(val_df)
test_df = model.transform_features(test_df)

model.fit(train_df.drop('Price', axis=1), train_df['Price'])
print(model.score_log(val_df.drop('Price', axis=1), np.exp(val_df['Price'])))